# Bungee Jumping Drop

Shreya Chowdhary & Gail Romer


In [20]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

# import functions from the modsim.py module
from modsim import *

In [79]:
m = UNITS.meter
s = UNITS.second
kg = UNITS.kilogram
N = UNITS.newton

<Unit('newton')>

We used google to find the average weight for a human, approximated the diameter of a spherical human by curling in balls and measuring ourselves, found k value for bungee cord from https://bungeejournal.academic.wlu.edu/files/2014/11/Does-a-Bungee-Cord-Behave-Like-an-Ideal-Spring.pdf

drag coefficient of a sphere: https://en.wikipedia.org/wiki/Drag_coefficient

mass bungee taken from the jump notebook (?)

In [80]:
params = Params(init_height = 220 * m,
                g = 9.8 * m/s**2,
                mass_human = 62 * kg,
                mass_object = 10 * kg,
                diameter = 0.8128 * m, # diameter of a spherical human
                rho = 1.2 * kg/m**3,   # rho of air for drag calculations
                k = 2.7289 * N/m,      # k value for an average bungee cord
                bungee_rest_length = 100 * m, # bungee cord less than half of init_height FOR NOW
                t_end = 20 * s,
               dt = 1*s,
               C_d = 0.47,
               mass_bungee = 75 * kg)

,values
init_height,220 meter
g,9.8 meter / second ** 2
mass_human,62 kilogram
mass_object,10 kilogram
diameter,0.8128 meter
rho,1.2 kilogram / meter ** 3
k,2.7289 newton / meter
bungee_rest_length,100 meter
t_end,20 second
dt,1 second


In [84]:
def make_system(params):
    init = State(t = 0 * s, 
                 h = params.init_height, 
                 v = 0 * m/s)
    area = np.pi * (diameter/2)**2
    mu = params.mass_bungee/(total_mass)
    system = System(params, init = init, area = area, mu = mu)
    return system

In [85]:
system = make_system(params)
free_fall = free_fall(system)

DimensionalityError: Cannot convert from 'kilogram * meter / second ** 2' ([length] * [mass] / [time] ** 2) to 'kilogram * meter ** 3 / second ** 2' ([length] ** 3 * [mass] / [time] ** 2)

In [71]:
def drag_force(v, system):
    unpack(system)
    return -np.sign(v) * -rho * v**2 * C_d * area / 2

In [61]:
def whip_force(system, state):
    unpack(system)
    t, h, v = state
    return (total_mass)*(mu * v**2/2)*(mu*(bungee_rest_length + h) + 2*bungee_rest_length)

In [11]:
def spring_force(h, system):
    unpack(system)
    return (init_height - h) - bungee_rest_length

In [30]:
def free_fall_event(state, t, system):
    unpack(system)
    t, h, v = state
    
    return bungee_rest_length - h

In [86]:
def slope_func_1(state, t, system):
    unpack(system)
    t, h, v = state
    
    f_drag = drag_force(v, system)
    f_whip = whip_force(system, state)
    f_grav = (total_mass)*(-g)
    net_force = f_drag + f_whip + f_grav
    print("net_force: " + net_force)
    a = (drag_force(v, system) + whip_force(system, state) + (total_mass)*(-g))/total_mass
    dhdt = v
    dvdt = a 
    
    return dhdt, dvdt

### Stage 1: Freefall

In [74]:
def free_fall(sys):
    system = System(sys, total_mass = sys.mass_human + sys.mass_object)
    slope_func_1(sys.init, 0 * s, sys)
#     free_fall_results = run_ode_solver(system, slope_func_1, events = free_fall_event, max_step = 0.5*s)
#     free_fall_state = State(h = get_last_label(results.h), v = get_last_label(results.v))

    #End characteristic of Stage 1 => (Bungee_rest_length - delta Y = 0)
    # return Final Height of Free Fall Stage, final V of free fall stage, final t of free fall stage

### Stage 2: Stretch

In [24]:
final_V_of_freefall = 20 * m/s
final_height_of_freefall = 100 * m
t = 20 * s

delta_Y = 220 * m - final_height_of_freefall


F_g = -(params.mass_human+params.mass_object)*params.g
F_bungee = -params.k*(delta_Y - params.bungee_rest_length) # NEED TO MAKE THIS A FUNC OF DT
# F_drag
a = -(F_bungee - F_g)/(params.mass_human+params.mass_object)
v = final_V_of_freefall + a*dt
height = final_height_of_freefall + v*dt


# EPE_of_Bungee = (1/2)*params.k*delta_Y

#End characteristic of Stage 2 => (v = 0)
# return Final Height of Stretch Stage, final V of stretch stage, final t of stretch stage, EPE of Bungee

<Quantity(163.73399999999998, 'newton')>

### Stage 3: Release
is this a necessary stage?

In [25]:
final_V_of_stretch = 0 * m/s
final_height_of_stretch = 10 * m
t = 40 * s
EPE_of_Bungee =  (1/2)*params.k*delta_Y

delta_Y = 220 * m - final_height_of_freefall

F_g = -(params.mass_human+params.mass_object)*params.g
F_bungee = -params.k*(delta_Y - params.bungee_rest_length)
a = -(F_bungee - F_g)/(params.mass_human+params.mass_object)
v = final_V_of_freefall + a*dt
height = final_height_of_freefall + v*dt

#End characteristic of Stage 2 => (v = 0)
# return Final Height of Stretch Stage, final V of stretch stage, final t of stretch stage

<Quantity(110.95802777777777, 'meter')>

### Stage 4: Return (pull)

### Stage 5: Freefly

### End Stage: Pass Height 0